<a href="https://colab.research.google.com/github/Paraskevi-KIvroglou/chatbot_from_hugging_face_with_langchain/blob/main/chatbot_from_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain transformers==4.31 bitsandbytes accelerate xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using

In [8]:
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import langchain.globals
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer,  BitsAndBytesConfig, pipeline
#import streamlit as st

my_model_id = "KvrParaskevi/Llama-2-7b-Hotel-Booking-Model" #os.getenv('MODEL_REPO_ID', 'Default Value')
token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

template = """<<SYS>>
You are an AI having conversation with a human. Below is an instruction that describes a task.
Write a response that appropriately completes the request.
Reply with the most helpful and logic answer. During the conversation you need to ask the user
the following questions to complete the hotel booking task.

1) Where would you like to stay and when?
2) How many people are staying in the room?
3) Do you prefer any ammenities like breakfast included or gym?
4) What is your name, your email address and phone number?

Make sure you receive a logical answer from the user from every question to complete the hotel
booking process.
<</SYS>>

Previous conversation:
{chat_history}

Human: {question}
AI:"""

#@st.cache_resource
def load_model():
    quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(my_model_id)
    model = AutoModelForCausalLM.from_pretrained(my_model_id, device_map="auto",quantization_config=quantization_config) #

    return tokenizer,model

#@st.cache_resource
def load_pipeline():
    tokenizer, model = load_model()
    pipe = pipeline("text-generation",
                    model= model,
                    tokenizer = tokenizer,
                    max_tokens = 50,top_k = 30, early_stopping=True,
                    temperature = 0.1,repetition_penalty = 1.03)

    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

# def generate_from_pipeline(text, pipe):
#     return pipe(text)

llm = load_pipeline()

def demo_miny_memory():
    prompt = PromptTemplate.from_template(template)
    memory = ConversationBufferMemory(memory_key="chat_history", llm = llm)
    return memory

def demo_chain(input_text, memory):
    conversation = ConversationChain(
        llm=llm,
        verbose=True,
        memory=memory
    )

    chat_reply = conversation(input=input_text)
    return chat_reply['response']

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at KvrParaskevi/Llama-2-7b-Hotel-Booking-Model and are newly initialized: ['model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.1.s

In [ ]:
import os
import streamlit as st
import chatbot as demo_chat
from transformers import AutoModelForCausalLM, AutoTokenizer

st.title("Hi, I am Chatbot Philio :mermaid:")
st.write("I am your hotel booking assistant for today.")

scrollable_div_style = """
<style>
.scrollable-div {
    height: 200px;  /* Adjust the height as needed */
    overflow-y: auto;  /* Enable vertical scrolling */
    padding: 5px;
    border: 1px solid #ccc;  /* Optional: adds a border around the div */
    border-radius: 5px;  /* Optional: rounds the corners of the border */
}
</style>
"""

def render_chat_history(chat_history):
    #renders chat history
    for message in chat_history:
        if(message["role"]!= "system"):
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

# def generate_response(chat_history):
#     tokenized_chat = tokenizer.apply_chat_template(chat_history, tokenize=True, add_generation_prompt=True, return_tensors="pt")
#     outputs = model.generate(tokenized_chat, do_sample =True, max_new_tokens=50, temperature = 0.3, top_p = 0.85)
#     answer = tokenizer.decode(outputs[0][tokenized_chat.shape[1]:],skip_special_tokens=True)
#     final_answer = answer.split("<")[0]
#     return final_answer

#Application
#Langchain memory in session cache
if 'memory' not in st.session_state:
    st.session_state.memory = demo_chat.demo_miny_memory()

system_content = """
You are an AI having conversation with a human. Below is an instruction that describes a task.
Write a response that appropriately completes the request.
Reply with the most helpful and logic answer. During the conversation you need to ask the user
the following questions to complete the hotel booking task.

1) Where would you like to stay and when?
2) How many people are staying in the room?
3) Do you prefer any ammenities like breakfast included or gym?
4) What is your name, your email address and phone number?

Make sure you receive a logical answer from the user from every question to complete the hotel
booking process.
"""
#Check if chat history exists in this session
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = [
        {
            "role": "system",
            "content": system_content,
        },
        {"role": "assistant", "content": "Hello, how can I help you today?"},
    ] #Initialize chat history

# if 'model' not in st.session_state:
#     st.session_state.model = model

st.markdown('<div class="scrollable-div">', unsafe_allow_html=True) #add css style to container
render_chat_history(st.session_state.chat_history)

#Input field for chat interface
if input_text := st.chat_input(placeholder="Here you can chat with our hotel booking model."):

    with st.chat_message("user"):
        st.markdown(input_text)
    st.session_state.chat_history.append({"role" : "user", "content" : input_text}) #append message to chat history

    with st.spinner("Generating response..."):
        first_answer = demo_chat.demo_chain(input_text, st.session_state.memory)

        with st.chat_message("assistant"):
            st.markdown(first_answer)
        st.session_state.chat_history.append({"role": "assistant", "content": first_answer})
st.markdown('</div>', unsafe_allow_html=True)
